In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for thier loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [ ]:
documents

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")

os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

llm = ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain_chroma import Chroma

vectorstore=Chroma.from_documents(documents, embedding=embeddings)

In [ ]:
vectorstore

In [ ]:
vectorstore.similarity_search("cat")

In [ ]:
# Async Query
await vectorstore.asimilarity_search("cat")

In [ ]:
vectorstore.similarity_search_with_score("cat")

In [ ]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat', 'dog'])

In [ ]:
retriver = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)

retriver.batch(['cat', 'dog'])

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain={"context":retriever, "question":RunnablePassthrough()} | prompt | llm

response=rag_chain.invoke("tell me about dogs")
print(response.content)